In [1]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
import numpy as np
from math import exp
import ast
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

In [3]:
# Initialize csv file paths
csv_file_path= './Dataset Archive/merged_data_mod_scores_filtered_US_release_year_star_scores.csv'
# Create DataFrames from both CSV files
df = pd.read_csv(csv_file_path)


In [7]:
df['Movie Contribution to Director and Production Scores'] = 0

#Calculates the contribution of a movie to a star score based on whether or not it was a top 100 domestic grossing movie of that year
release_years = range(2010, 2024)
#iterates through the years
for year in release_years:
    df_year = df[df["Release Year"] == year]
    print(df_year.head())
    #sorts by revenue (descending)
    df_year = df_year.sort_values(by=['Merged Revenue'], ascending=False)
    #iterate through top 100 movies, give them points based on the ranking (100 to the top grossing, 99 to the 2nd top, ..., 1 to the 100th)
    for i in range(0,100):
        tmdb_id_to_update = df_year['IMDB ID'].iloc[i]
        #record the contribution in the dataframe
        df.loc[df['IMDB ID'] == tmdb_id_to_update, 'Movie Contribution to Director and Production Scores'] = 100 - i

      Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0 Running Time  \
4453          4453          4453        9232  1 hr 44 min   
4454          4454          4454        9234  1 hr 40 min   
4455          4455          4455        9236  1 hr 54 min   
4456          4456          4456        9237  1 hr 57 min   
4457          4457          4457        9238  1 hr 43 min   

                                                 Genres  \
4453                             Drama Mystery Thriller   
4454  Adventure Animation Comedy Family Fantasy Musi...   
4455                                             Comedy   
4456                    Action Adventure Crime Thriller   
4457          Adventure Animation Comedy Family Fantasy   

                      IMDB Title   MPAA                 Domestic Distributor  \
4453          In My Sleep (2010)  PG-13                Morning Star Pictures   
4454              Tangled (2010)     PG  Walt Disney Studios Motion Pictures   
4455  Dinner for Schmucks (2010)  PG-1

In [8]:
# Create a DataFrame with unique production company IDs
unique_dist_ids = list(set(df['Domestic Distributor ID']))

# Create a list of years from 2011 to 2023
years = [str(year) for year in range(2010, 2024)]

# Initialize the data with zeros
data = {f'star_{year}': [0] * len(unique_dist_ids) for year in years}
data['ids'] = unique_dist_ids

# Create the 'output_df' DataFrame
dist_df = pd.DataFrame(data)

# Print the first few rows of the 'output_df' DataFrame for debugging
print(dist_df.head())

   star_2010  star_2011  star_2012  star_2013  star_2014  star_2015  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   star_2016  star_2017  star_2018  star_2019  star_2020  star_2021  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   star_2022  star_2023  ids  
0          0          0    0  
1          0          0    1  
2          0          0    2  
3          0          

In [9]:
import pandas as pd

#Iterate through main data
for index, row in df.iterrows():
    dist_ids = row['Domestic Distributor ID']
    # Iterate through each dist_id in the dist_ids list
    for dist_id in [dist_ids]:
        # Your existing code...
        release_year = row['Release Year']
        score_contribution = row['Movie Contribution to Director and Production Scores']
        # Iterate through each dist_id in the dist_ids array
        dist_df_row = dist_df[dist_df['ids'] == dist_id]
        if not dist_df_row.empty:
            dist_score = dist_df_row[f'star_{release_year}'].values[0]
            # Add the contribution from the movie to the total star score for that company for that year
            if not pd.isna(score_contribution):
                dist_score += score_contribution
            # Assign the updated production company score to the corresponding 'star_yyyy' column
            dist_df.loc[dist_df['ids'] == dist_id, f'star_{release_year}'] = dist_score

# Sum the star scores for the previous three years (beginning in 2012)
output_dists = dist_df.copy(deep=True)

for dist_id in dist_df['ids']:
    for column in dist_df.columns:
        if column.startswith("star_"):
            release_year = int(column.split("_")[1])
            if release_year > 2011:
                previous_year = release_year - 1
                year_before_previous = release_year - 2
                dist_df_row = dist_df[dist_df['ids'] == dist_id]
                if not dist_df_row.empty:
                    dist_score = dist_df_row[column].values[0]
                    # Calculate score from the previous year
                    previous_year_score = dist_df_row[f'star_{previous_year}'].values[0]
                    dist_score += previous_year_score
                    # Calculate score from the year before the previous year
                    year_before_previous_score = dist_df_row[f'star_{year_before_previous}'].values[0]
                    dist_score += year_before_previous_score
                    # Assign the updated director score to the corresponding 'star_year' column
                    output_dists.loc[output_dists['ids'] == dist_id, column] = dist_score


In [10]:
print(output_dists.head())

   star_2010  star_2011  star_2012  star_2013  star_2014  star_2015  \
0         74         81        178        167        183        201   
1        585        562       1751       1760       2299       2433   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   star_2016  star_2017  star_2018  star_2019  star_2020  star_2021  \
0        220        209        168         86        554        656   
1       2699       2322       2046       1560        915        453   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   star_2022  star_2023  ids  
0        690        153    0  
1         79          0    1  
2          0          0    2  
3          0          

In [11]:
output_dists.to_csv('domestic_dist_df.csv', index=False)

In [12]:
# Assuming df is your DataFrame
df_output = df.copy()  # Copy the original DataFrame
df_output = df_output[df_output['Release Year'] > 2012]
df_output['Domestic Distributor Score'] = 0  # create an empty star scores column

for index, row in df_output.iterrows():
    row_dist_info = row['Domestic Distributor ID']  # Extract prod_ids array
    if isinstance(row_dist_info, int) and row_dist_info != 0:
        row_dist_info = [row_dist_info]
    else:
        row_dist_info = []  # Set to an empty list if not an integer or if it's 0
    # computes the sum of production company scores
    for dist_id in row_dist_info:
        release_date = row['Release Year']
        previous_year = release_date - 1
        dist_info_row = output_dists[output_dists['ids'] == dist_id]  # Get the row containing the queried prod id
        star_column_name = f'star_{previous_year}'  # get the star score column for the previous year
        star_score = dist_info_row[star_column_name].values[0]  # Use the previous year as the star score
        if star_score is not None:
            df_output.loc[df['IMDB ID'] == row['IMDB ID'], 'Domestic Distributor Score'] += star_score

# Save to csv
print(df_output.head())

   Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0 Running Time  \
0             0             0           0  1 hr 22 min   
1             1             1           2  1 hr 49 min   
2             2             2           3  1 hr 30 min   
3             3             3           4  1 hr 46 min   
4             4             4           6  1 hr 29 min   

                                           Genres               IMDB Title  \
0                               Documentary Sport     Into the Mind (2013)   
1             Action Crime Drama Mystery Thriller       Broken City (2013)   
2                                           Drama   Redwood Highway (2013)   
3                     Biography Drama History War  Bless Me, Ultima (2012)   
4  Action Adventure Comedy Horror Sci-Fi Thriller           Spiders (2013)   

    MPAA      Domestic Distributor  Domestic Opening  \
0    NaN                     Empty               NaN   
1      R     Twentieth Century Fox         8268908.0   
2  PG-13      

In [13]:
df_output.to_csv('./Dataset Archive/merged_data_mod_scores_filtered_US_release_year_star_scores_dist_scores.csv', index=False)